<a href="https://colab.research.google.com/drive/18gghD8RqO4zoZKft5WhhnhNOESCKUwWI?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Multi-Agent Debate

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [3]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [4]:
genai.configure(api_key=API_KEY)

In [5]:
class DebateAgent:
    """Virtual agent with specific perspective"""
    def __init__(self, name, perspective, model):
        self.name = name
        self.perspective = perspective
        self.model = model
        self.current_position = None

    def propose_answer(self, question):
        """Step 1: Propose initial answer"""
        prompt = f"""You are {self.name}, a virtual agent with this perspective: {self.perspective}

        Question: {question}

        From your perspective, provide your answer and reasoning:"""

        response = self.model.generate_content(prompt).text
        self.current_position = response.strip()
        return self.current_position

    def critique_others(self, question, other_positions):
        """Step 2: Critique other agents' positions"""
        others_text = "\n\n".join([
            f"{agent_name}'s position:\n{position}"
            for agent_name, position in other_positions.items()
        ])

        prompt = f"""You are {self.name} ({self.perspective})

        Question: {question}

        Other agents' positions:
        {others_text}

        Critically analyze their positions. Identify flaws, biases, or weaknesses:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def refine_position(self, question, critiques_received):
        """Step 3: Refine position based on critiques"""
        critiques_text = "\n\n".join([
            f"Critique from {agent_name}:\n{critique}"
            for agent_name, critique in critiques_received.items()
        ])

        prompt = f"""You are {self.name} ({self.perspective})

        Question: {question}

        Your current position:
        {self.current_position}

        Critiques you received:
        {critiques_text}

        Revise your position, addressing valid critiques:"""

        response = self.model.generate_content(prompt).text
        self.current_position = response.strip()
        return self.current_position

class MultiAgentDebate:
    def __init__(self, num_agents=3):
        self.model = genai.GenerativeModel("gemini-2.0-flash-exp")
        self.agents = []
        self.debate_history = []

    def add_agent(self, name, perspective):
        """Add a virtual agent to the debate"""
        agent = DebateAgent(name, perspective, self.model)
        self.agents.append(agent)

    def debate(self, question, rounds=2):
        """Main debate process"""
        print(f"\n{'='*60}")
        print(f"🗣️  Multi-Agent Debate")
        print(f"{'='*60}")
        print(f"Question: {question}\n")

        print(f"Participants: {len(self.agents)} virtual agents\n")
        for agent in self.agents:
            print(f"  • {agent.name}: {agent.perspective}")
        print()

        # Initial proposals
        print(f"{'='*60}")
        print(f"ROUND 1: Initial Proposals")
        print(f"{'='*60}\n")

        for agent in self.agents:
            print(f"👤 {agent.name}:")
            position = agent.propose_answer(question)
            print(f"{position}\n")

        # Debate rounds
        for round_num in range(2, rounds + 2):
            print(f"{'='*60}")
            print(f"ROUND {round_num}: Critique & Refinement")
            print(f"{'='*60}\n")

            # Critique phase
            print(f"{'─'*60}")
            print(f"Critique Phase")
            print(f"{'─'*60}\n")

            all_critiques = {}

            for agent in self.agents:
                # Get other agents' positions
                other_positions = {
                    a.name: a.current_position
                    for a in self.agents if a != agent
                }

                print(f"🔍 {agent.name} critiquing others...")
                critique = agent.critique_others(question, other_positions)
                all_critiques[agent.name] = critique
                print(f"{critique[:150]}...\n")

            # Refinement phase
            print(f"{'─'*60}")
            print(f"Refinement Phase")
            print(f"{'─'*60}\n")

            for agent in self.agents:
                # Get critiques directed at this agent
                critiques_for_agent = {
                    name: critique
                    for name, critique in all_critiques.items()
                    if name != agent.name
                }

                print(f"✨ {agent.name} refining position...")
                refined = agent.refine_position(question, critiques_for_agent)
                print(f"{refined[:150]}...\n")

        # Consensus building
        print(f"{'='*60}")
        print(f"FINAL PHASE: Consensus Building")
        print(f"{'='*60}\n")

        final_answer = self.build_consensus(question)

        print(f"{'='*60}")
        print(f"💡 CONSENSUS ANSWER")
        print(f"{'='*60}")
        print(final_answer)
        print()

        return final_answer

    def build_consensus(self, question):
        """Step 4: Build consensus from refined positions"""
        all_positions = "\n\n".join([
            f"{agent.name}'s final position:\n{agent.current_position}"
            for agent in self.agents
        ])

        prompt = f"""Question: {question}

        Final positions from all agents:
        {all_positions}

        Synthesize these positions into a single, well-reasoned answer that:
        1. Incorporates the strongest arguments
        2. Addresses critiques and weaknesses
        3. Provides the most factual and logical conclusion

        Consensus answer:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

In [ ]:
# Example 1: Fact-Checking
print("="*60)
print("EXAMPLE 1: Fact-Checking Through Debate")
print("="*60)

debate1 = MultiAgentDebate()
debate1.add_agent("Skeptic", "Questions claims and demands evidence")
debate1.add_agent("Researcher", "Provides data-driven analysis")
debate1.add_agent("Synthesizer", "Integrates multiple perspectives")

debate1.debate(
    "Is coffee consumption good or bad for health?",
    rounds=2
)


# Example 2: Complex Reasoning
print("\n" + "="*60)
print("EXAMPLE 2: Complex Policy Question")
print("="*60)

debate2 = MultiAgentDebate()
debate2.add_agent("Economist", "Analyzes economic impacts and costs")
debate2.add_agent("Environmentalist", "Prioritizes ecological sustainability")
debate2.add_agent("Pragmatist", "Focuses on practical implementation")

debate2.debate(
    "Should cities ban gas-powered cars by 2030?",
    rounds=2
)


# Example 3: Ethical Reasoning
print("\n" + "="*60)
print("EXAMPLE 3: Ethical Dilemma")
print("="*60)

debate3 = MultiAgentDebate()
debate3.add_agent("Ethicist", "Applies moral philosophy and principles")
debate3.add_agent("Consequentialist", "Evaluates outcomes and consequences")
debate3.add_agent("Rights Advocate", "Prioritizes individual rights and freedoms")

debate3.debate(
    "Should AI-generated art be copyrightable?",
    rounds=2
)


# Example 4: Scientific Hypothesis
print("\n" + "="*60)
print("EXAMPLE 4: Scientific Debate")
print("="*60)

debate4 = MultiAgentDebate()
debate4.add_agent("Experimentalist", "Values empirical evidence and testing")
debate4.add_agent("Theorist", "Focuses on theoretical consistency")
debate4.add_agent("Critic", "Identifies methodological weaknesses")

debate4.debate(
    "Is dark matter the best explanation for galaxy rotation curves?",
    rounds=2
)


# Example 5: Legal Reasoning
print("\n" + "="*60)
print("EXAMPLE 5: Legal Argument")
print("="*60)

debate5 = MultiAgentDebate()
debate5.add_agent("Prosecutor", "Argues for strict liability")
debate5.add_agent("Defense", "Argues for protection of rights")
debate5.add_agent("Judge", "Seeks balanced interpretation of law")

debate5.debate(
    "Should social media companies be liable for user-generated content?",
    rounds=2
)


# Example 6: Business Decision
print("\n" + "="*60)
print("EXAMPLE 6: Business Strategy Debate")
print("="*60)

debate6 = MultiAgentDebate()
debate6.add_agent("Growth Strategist", "Prioritizes expansion and market share")
debate6.add_agent("Risk Manager", "Focuses on risk mitigation and stability")
debate6.add_agent("Customer Advocate", "Emphasizes customer value and satisfaction")

debate6.debate(
    "Should our startup pursue rapid growth through venture funding or slow organic growth?",
    rounds=2
)


# Example 7: Reducing Hallucinations
print("\n" + "="*60)
print("EXAMPLE 7: Fact Verification")
print("="*60)

debate7 = MultiAgentDebate()
debate7.add_agent("Fact Checker", "Verifies claims against known facts")
debate7.add_agent("Devil's Advocate", "Challenges assumptions and claims")
debate7.add_agent("Neutral Observer", "Seeks objective truth")

debate7.debate(
    "What are the proven health benefits of intermittent fasting?",
    rounds=2
)


print("✅ Multi-Agent Debate Complete!")

EXAMPLE 1: Fact-Checking Through Debate

🗣️  Multi-Agent Debate
Question: Is coffee consumption good or bad for health?

Participants: 3 virtual agents

  • Skeptic: Questions claims and demands evidence
  • Researcher: Provides data-driven analysis
  • Synthesizer: Integrates multiple perspectives

ROUND 1: Initial Proposals

👤 Skeptic:
Alright, another health claim. Let's not jump to conclusions here. The question is whether coffee consumption is *good or bad* for health. That's a pretty broad statement, implying a universal truth.

First, we need to define "coffee consumption." Are we talking about a single espresso a day, or a pot of black coffee? Are we adding sugar, cream, or artificial sweeteners? The answer likely changes drastically depending on the quantity and preparation methods.

Second, "good or bad" is a gross oversimplification. Health outcomes are complex and multifaceted. Coffee consumption could be beneficial for one person and detrimental to another. Factors like ge